### Week 3 Lab/Peer-Graded Assignment: Brandon Galloway  
This Notebook contains my submission for the week 3 peer-graded assesment. I've labeled each section with markdown header to hopefully give you an easier time reading through the code and have labeled each gradable question's output with an above markdown.I hope you find everything well. Thanks for reviewing my submission!

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import folium

from bs4 import BeautifulSoup






In [2]:
#Goal: Explore and cluster the neighborhoods in Toronto

In [3]:
#Create Empty Pandas DF
df_tor = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
df_tor

,PostalCode,Borough,Neighborhood


# Parsing Wikipedia  
To first get the data we utalize BeautifulSoup to grab data from the wikipedia url. This is first passed through requests and then into soup to get a soup object. We can then grab the wikitable component from the source and parse each table entry into our df_tor DataFrame

In [4]:
addr = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(addr).text
soup = BeautifulSoup(source,'lxml')
t_table = soup.find('table',class_='wikitable')

In [5]:
for i in t_table.tbody.find_all('tr')[1:]:
    content = []
    for j in i.find_all('td'):
        content.append(j.text)    
    df_tor = df_tor.append(dict(zip(df_tor.columns,content)),ignore_index=True)
df_tor

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


# Slicing Column  
We now take the data and sline the new line data off of our Neighborhood Column. This gets our dataset ready for the upcoming, deeper cleaning steps. A copy of the df_tor is made called df_tor_data to create 2 seperate csv files of the data.

In [6]:
df_tor_data = df_tor
df_tor_data["Neighborhood"] = df_tor_data["Neighborhood"].str.replace('\n','')

In [7]:
#df_tor_data.to_csv(r'D:\desktop\outcomes\toronto.csv')
df_tor_data

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


# Grouping Postal Codes  
Here we create a groupby "codes" that contains the indexes of each apperance of each postal code. This will be used later to find combined data.

In [8]:
codes = df_tor_data.groupby(df_tor_data["PostalCode"]).groups

# Cleaning The Dataset  
Here we create another empty DataFrame and ,for each code in our codes groupby, we set the Neighborhoods to the distict set of Neighborhoods for that postal code.

In [9]:
df_clean = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
df_clean

,PostalCode,Borough,Neighborhood


In [10]:
for post in codes.keys():
    content = [post]
    content.append(df_tor_data.iloc[codes[post][0]][1])
    nb = []
    for i in codes[post]:  
        #i = PostalCode
        #Borough = index [1]
        #Neighborhoods = set() on a list we keep updated
        nb.append(df_tor_data.iloc[i][2])
        #print(df_tor_data.iloc[i])
        nb = list(set(nb))
        nbf = ', '.join(nb)
    if(nbf != "Not assigned"):
        content.append(nbf)
    else:
        content.append(content[1])
    if(content[1] != "Not assigned"):
        df_clean = df_clean.append(dict(zip(df_clean.columns,content)),ignore_index=True)    

# Question #1  
This is the result required in question #1 for the peer lab

In [11]:
df_clean
#df_clean.to_csv(r'D:\desktop\outcomes\toronto clean.csv')

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Morningside, West Hill, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


## Using Geocoder To Grab Latitude and Longitude  
Now that we have our cleaned Wikipedia Toronto Dataset, we can begin to gather the location data we need to utalize the foursquare api. The getLatLong function returns a tuple of the latitude and longitude from the geocoder output which is then fed into the DataFrame.

In [12]:
def getLatLong(postal_code):
    coords = None
    while(coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        coords = g.latlng
        print(g.latitude,g.longitude)

In [13]:
#for i in range(df_geo.shape[0]):  
    #geodata = getLatLong(df_geo.iloc[i]["PostalCode"])  
    #df_geo.iat[i,3] = geodata[0]
    #df_geo.iat[i,4] = geodata[1]
#df_geo

# Geolibrary Problems  
Throughout my testing I, unfortunatly, could not get the geocoder library to produce anything but a None output, even over the course of an hour-long test. This was mentioned as a potential issue in the assignment and thus I have loaded the provided .csv file to continue to clustering. Theoretically the code above would produce the correct output. Thank you for your understanding.

In [14]:

coords = pd.read_csv("https://cocl.us/Geospatial_data")

In [15]:
df_geo = df_clean
df_geo["Latitude"] = ""
df_geo["Longitude"] = ""

# Question #2  
The DataFrame displayed below contains the answer to peer-lab question #2

# Latitude And Longitude Formatted  
Here we have read the .csv and popuated new columns with the latitude and longitude data for our Dataset. This data is referred to as df_geo

In [16]:
for i in range(df_geo.shape[0]):
    df_geo.iat[i,3] = coords.iloc[i]["Latitude"]
    df_geo.iat[i,4] = coords.iloc[i]["Longitude"]
df_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Morningside, West Hill, Guildwood",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",43.7111,-79.2846
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.6927,-79.2648


# Clustering  
To begin clustering I have first displayed the geolocation data we have obtained onto a folium map

In [17]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Borough'], df_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

!["Clustered_Map](https://i.imgur.com/LgmxI5V.png)

# Foursquare Developer ID  
For security reasons the CLIENT_ID and CLIENT_SECRET have been removed from this notebook. The following code segments will not run without them but the outputs should still be visable.

In [34]:
CLIENT_ID = 'Client_ID'
CLIENT_SECRET = 'Client_Secret'
VERSION = '20180605'

# Data Encoding For Clustering  
Here we will follow the steps from the New York Dataset lab and get the foursquare venue data from areas surrounding our neighborhoods. Then we will one-hot encode that data for use in clustering

In [19]:
radius = 500
LIMIT = 100
# Functions From The Labs
#extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
# type your answer here

toronto_venues = getNearbyVenues(names=df_geo['Neighborhood'],
                                   latitudes=df_geo['Latitude'],
                                   longitudes=df_geo['Longitude']
                                  )



Malvern, Rouge
Rouge Hill, Highland Creek, Port Union
Morningside, West Hill, Guildwood
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Golden Mile, Oakridge, Clairlea
Cliffside, Cliffcrest, Scarborough Village West
Cliffside West, Birch Cliff
Dorset Park, Wexford Heights, Scarborough Town Centre
Maryvale, Wexford
Agincourt
Tam O'Shanter, Sullivan, Clarks Corners
Agincourt North, Steeles East, L'Amoreaux East, Milliken
L'Amoreaux West
Upper Rouge
Hillcrest Village
Henry Farm, Fairview, Oriole
Bayview Village
York Mills, Silver Hills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Wilson Heights, Downsview North, Bathurst Manor
Northwood Park, York University
Downsview East, CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

# Venue Data  
Here we have a DataFrame of the collected venues and their attached Neighborhood

In [21]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Highland Creek, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Highland Creek, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Morningside, West Hill, Guildwood",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Morningside, West Hill, Guildwood",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [24]:
toronto_onehot.shape

(2238, 272)

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [26]:
toronto_grouped.shape

(99, 272)

# Encoded Data  
We now have our grouped encoded data and are ready to begin clustering our neighborhoods!

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Electronics Store,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Drugstore
1,"Agincourt North, Steeles East, L'Amoreaux East...",Park,Playground,Asian Restaurant,College Stadium,Colombian Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,Berczy Park,Coffee Shop,Cocktail Bar,Café,Steakhouse,Beer Bar,Farmers Market,Seafood Restaurant,Italian Restaurant,Bakery,Restaurant
4,"Bloordale Gardens, Old Burnhamthorpe, Markland...",Liquor Store,Pizza Place,Café,Convenience Store,Beer Store,Pharmacy,Comic Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant


In [29]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Electronics Store,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Drugstore
1,"Agincourt North, Steeles East, L'Amoreaux East...",Park,Playground,Asian Restaurant,College Stadium,Colombian Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,Berczy Park,Coffee Shop,Cocktail Bar,Café,Steakhouse,Beer Bar,Farmers Market,Seafood Restaurant,Italian Restaurant,Bakery,Restaurant
4,"Bloordale Gardens, Old Burnhamthorpe, Markland...",Liquor Store,Pizza Place,Café,Convenience Store,Beer Store,Pharmacy,Comic Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant
5,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden Center,Light Rail Station,Brewery,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Restaurant,Garden
6,Caledonia-Fairbanks,Park,Fast Food Restaurant,Pharmacy,Market,Women's Store,Golf Course,Gluten-free Restaurant,Greek Restaurant,Eastern European Restaurant,Dumpling Restaurant
7,Canada Post Gateway Processing Centre,Hotel,Coffee Shop,Gym / Fitness Center,Mediterranean Restaurant,Middle Eastern Restaurant,Burrito Place,Sandwich Place,American Restaurant,Fried Chicken Joint,Donut Shop
8,Cedarbrae,Athletics & Sports,Bank,Caribbean Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Hakka Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Sushi Restaurant,Chinese Restaurant,Middle Eastern Restaurant,Bakery,Spa


In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 0, 1, 1, 1, 2, 1, 1, 1])

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_geo[["Borough","Neighborhood","Latitude","Longitude"]]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",43.8067,-79.1944,1.0,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.7845,-79.1605,1.0,Bar,History Museum,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,Scarborough,"Morningside, West Hill, Guildwood",43.7636,-79.1887,1.0,Electronics Store,Rental Car Location,Spa,Intersection,Pizza Place,Mexican Restaurant,Breakfast Spot,Medical Center,Discount Store,Dim Sum Restaurant
3,Scarborough,Woburn,43.771,-79.2169,1.0,Coffee Shop,Korean Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,Scarborough,Cedarbrae,43.7731,-79.2395,1.0,Athletics & Sports,Bank,Caribbean Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Hakka Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,Scarborough,Scarborough Village,43.7447,-79.2395,4.0,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262,1.0,Bus Station,Coffee Shop,Discount Store,Chinese Restaurant,Department Store,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Dog Run
7,Scarborough,"Golden Mile, Oakridge, Clairlea",43.7111,-79.2846,1.0,Bakery,Park,Intersection,Fast Food Restaurant,Metro Station,Bus Line,Bus Station,Soccer Field,Convenience Store,Comic Shop
8,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.7163,-79.2395,1.0,Motel,Movie Theater,American Restaurant,Women's Store,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
9,Scarborough,"Cliffside West, Birch Cliff",43.6927,-79.2648,1.0,Café,General Entertainment,College Stadium,Skating Rink,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [32]:
toronto_merged.dropna(inplace=True)

# Question #3  
The Folium map below shows the clustering data for question 3 from the peer lab.  
From attempting different numbers of clusters it appears that the majority of neighborhoods in the city center are very similar but groups emerge when we begin to explore neighborhoods to the northwest. There also was also a consistant difference few eastern neighborhoods that generated their own cluster due to placements near cemetaries and large green spaces that impacted the breakdown of local buisnesses.

In [33]:
# create map
map_clusters = folium.Map(location=[43.7232, -79.3832], zoom_start=10.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-len(rainbow)],
        fill=True,
        fill_color=rainbow[int(cluster)-len(rainbow)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

!["Clustered_Map](https://i.imgur.com/nMeoHc8.png)

##### Thank you!